##SinthIA: A Consensus-Based Dialogue System

SinthIA is an advanced dialogue system designed to generate consensus-based responses by querying multiple AI models, including GPT-4 and Llama. It intelligently analyzes the responses from these models, synthesizes the best parts, and creates a final, unified response. This process ensures that the output is well-rounded and reflects a balanced perspective, making it ideal for applications in decision-making support, conflict resolution, and collaborative problem-solving.

Key Features:
Multiple AI Model Integration: Utilizes both GPT-4 and Llama for generating diverse responses.
Consensus Building: Synthesizes responses from different models to create a consensus-based output.
Customizable Iterations: Configurable process for refining responses through multiple iterations.
Versatile Applications: Useful for decision-making tasks, advice generation, and conflict resolution.

In [ ]:
%pip install python-dotenv openai==0.28

import openai
import requests
from typing import Optional
import json

class SinthIA:
    def __init__(self, config_file="config.json", max_iterations=3):
        # Cargar claves desde archivo JSON
        self.config = self._load_config(config_file)
        self.openai_api_key = self.config["openai_api_key"]
        self.llama_api_key = self.config["llama_api_key"]
        self.llama_endpoint = "https://api.groq.com/openai/v1/chat/completions"

        self.max_iterations = max_iterations

        # Configuración de OpenAI
        openai.api_key = self.openai_api_key

    @staticmethod
    def _load_config(config_file: str) -> dict:
        """Carga las claves desde un archivo JSON."""
        try:
            with open(config_file, "r") as file:
                return json.load(file)
        except FileNotFoundError:
            raise FileNotFoundError(f"Configuration file not found: {config_file}")
        except json.JSONDecodeError:
            raise ValueError(f"The file {config_file} does not have a valid JSON format.")

    def query_gpt(self, prompt: str) -> str:
        """Consulta a GPT-4"""
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a neutral assistant focused on finding consensus."},
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message['content']
        except Exception as e:
            print("Error querying GPT.:", e)
            return "Error getting response from GPT."

    def query_llama(self, prompt: str) -> str:
        """Consulta a Llama 3"""
        headers = {
            "Authorization": f"Bearer {self.llama_api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": "llama3-70b-8192",
            "messages": [
                {"role": "system", "content": "You are a neutral assistant focused on finding consensus.."},
                {"role": "user", "content": prompt}
            ]
        }

        try:
            response = requests.post(self.llama_endpoint, headers=headers, json=payload)
            response.raise_for_status()  # Verifica si hubo errores HTTP
            return response.json()['choices'][0]['message']['content']
        except requests.RequestException as e:
            print("Error querying Llama.:", e)
            return "Error getting response from Llama."

    def analyze_responses(self, gpt_response: str, llama_response: str) -> str:
        """Analyze and synthesize the responses."""
        mediation_prompt = f"""
        You have two different responses:

        GPT's response: {gpt_response}

        Llama's response: {llama_response}

        Analyze both responses. Identify the strengths of each and create
        a consensus-based response that integrates the best aspects of both.
        """
        return self.query_gpt(mediation_prompt)

    def dialogue(self, initial_prompt: str) -> str:
        """"Main dialogue process."""
        print("🤖 Starting SinthIA dialogue....")

        # Respuestas iniciales
        gpt_response = self.query_gpt(initial_prompt)
        llama_response = self.query_llama(initial_prompt)

        for iteration in range(self.max_iterations):
            print(f"\n--- Iteración {iteration + 1} ---")
            print("Respuesta GPT:", gpt_response)
            print("Respuesta Llama:", llama_response)

            # Encuentra consenso
            consensus = self.analyze_responses(gpt_response, llama_response)
            print("Consenso:", consensus)

            # Condición de parada
            if self._consensus_reached(gpt_response, llama_response, consensus):
                return consensus

            # Actualiza las respuestas
            gpt_response = self.query_gpt(consensus)
            llama_response = self.query_llama(consensus)

        return consensus

    def _consensus_reached(self, gpt_resp: str, llama_resp: str, consensus: str) -> bool:
        """Basic consensus check."""
        return len(consensus) > len(gpt_resp) and len(consensus) > len(llama_resp)

def main():
    sinthia = SinthIA()
    prompt = "Recommendations for training in GenAI"
    resultado_final = sinthia.dialogue(prompt)
    print("\n🏁 Resultado Final:", resultado_final)

if __name__ == "__main__":
    main()
